In [1]:
import torch
import torch._dynamo
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
device

device(type='cuda')

In [2]:
def knn(x,k):
    rows = x.shape[0]
    idx = torch.arange(0,rows)
    comb = torch.combinations(idx,2)

    norms = torch.zeros(comb.shape[0])
    for i, pair in enumerate(comb):
        norms[i] = torch.linalg.vector_norm(x[pair[0]] - x[pair[1]])
    return torch.topk(-norms, k).values.negative()

In [3]:
k = 2
x = torch.randn(1000,50)
%time knn(x,k)

CPU times: user 7.56 s, sys: 58.6 ms, total: 7.62 s
Wall time: 7.46 s


tensor([5.2996, 5.6529])

In [4]:
def neighbor_dist(x):
    rows = x.shape[0]
    idx = torch.arange(0,rows)
    nchoose2 = int(0.5*rows*(rows-1))
        
    dist_squared = torch.zeros(nchoose2).to(device)
    
    counter=0
    for i in range(rows):
        for j in range(i,rows):
            if i == j:
                continue
            dist_squared[counter] = torch.sum((x[i] - x[j])**2)
            counter+=1
    return dist_squared

In [5]:
torch._dynamo.reset()
# torch._dynamo.config.dynamic_shapes = True
neighbor_dist_opt = torch.compile(neighbor_dist, backend="inductor", 
                        options={'trace.enabled':False})

In [6]:
x = torch.randn(1000,50)

In [ ]:
%%time
dist_squared = neighbor_dist_opt(x.detach().to(device))
torch.topk(-dist_squared, k).values.negative()

In [15]:
neighbor_dist_opt(x.detach().to(device))


tensor([ 79.5146, 104.3157,  68.3868,  ..., 120.3970,  78.6523, 109.0322],
       device='cuda:0')